<a href="https://colab.research.google.com/github/dtoralg/IE_Calidad_ML/blob/main/Ejercicios/Modulo%206/Modulo_6_Ejercicio_1_MLP_SECOM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **Ejercicio 1: Introducción al Deep Learning en Clasificación Binaria Industrial**
## Aplicación de una red neuronal multicapa (MLP) al dataset SECOM



## Introducción

En este ejercicio vamos a construir y comparar modelos clásicos de Machine Learning con una red neuronal multicapa (MLP) para resolver un problema real de clasificación binaria en un entorno industrial. Utilizaremos el **SECOM Manufacturing Data Set**, que contiene medidas de sensores en un proceso de manufactura y una variable objetivo que indica si se detectó un defecto o no.

Este tipo de problema es habitual en entornos industriales donde se requiere automatizar la identificación de productos defectuosos, y el uso de redes neuronales puede aportar mejoras en la capacidad de detección frente a modelos clásicos, especialmente cuando existen relaciones no lineales entre las variables.

**Lo que aprenderás en este ejercicio:**
- Preprocesar datos reales con ruido y valores nulos.
- Entrenar y evaluar una red neuronal multicapa (MLP) usando Keras.
- Comparar su rendimiento frente a modelos clásicos como Regresión Logística y Random Forest.
- Interpretar las curvas de aprendizaje y los resultados obtenidos.

**Licencia del dataset:** SECOM Manufacturing Data Set — Fuente: UCI ML Repository, Licencia: [CC BY 4.0](https://creativecommons.org/licenses/by/4.0/)

> ### Recomendación:
> Las redes neuronales hacen millones de operaciones matemáticas (multiplicaciones de matrices, derivadas, etc.) durante el entrenamiento. Una CPU puede hacer estas operaciones, pero una GPU está diseñada específicamente para hacerlas en paralelo y mucho más rápido.
>
> Por ello te recomiendo que para estos ejercicios cambies tu entorno de ejecución en Colab.
>
> Puedes hacerlo haciendo click en `Entorno de Ejecución` en la barra superior, después `Cambiar tipo de entorno de ejecución` y selecciona `GPU`. Esto hará que tus notebook ejecuten más rápido los cálculos con Deep Learning.


## Carga de librerías y configuración del entorno

In [ ]:

# Celda 1
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import confusion_matrix, classification_report, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import precision_recall_curve
from sklearn.utils import class_weight
from tensorflow.keras.callbacks import EarlyStopping
from imblearn.over_sampling import SMOTE
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
import requests
import zipfile
import io

# Configuración visual
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (10, 6)


## Descripción del dataset

In [ ]:
zip_url = "https://github.com/dtoralg/IE_Calidad_ML/raw/refs/heads/main/Data/Modulo%206/secom.zip"

# Descargar el zip
response = requests.get(zip_url)

# Verificar que la descarga fue exitosa
if response.status_code == 200:
    # Leer el archivo ZIP en memoria
    zip_file = zipfile.ZipFile(io.BytesIO(response.content))

    # Mostrar el contenido del ZIP
    print("Archivos contenidos:")
    print(zip_file.namelist())

    # Extraer todos los archivos (opcional)
    zip_file.extractall("secom_data")  # Carpeta donde se guardarán
    print("Archivos extraídos en la carpeta 'secom_data'.")
else:
    print("Error al descargar el archivo:", response.status_code)


In [ ]:
# Celda 2 - Cargar y preparar features y etiquetas
import pandas as pd

# Rutas de los archivos
features_url = '/content/secom_data/secom.data'
labels_url = '/content/secom_data/secom_labels.data'

# Cargar las features
X = pd.read_csv(..., sep='\s+', header=None)
# Renombrar columnas solo si el número coincide
X.columns = [f'var_{i}' for i in range(X.shape[1])]

# Cargar las etiquetas y fechas
y = pd.read_csv(..., sep='\s+', header=None)
y.columns = ['label', 'timestamp']  # Añadir nombres de columna
y['label'] = ... # Reemplaza los valores de y por valores binarios 0,1


# Verificar
print("Dimensiones de X:", X.shape)
...

In [ ]:
print("Dimensiones de y:", y.shape)
...

## Preprocesamiento de datos

In [ ]:
# Celda 3
# Verificar nulos
...

In [ ]:
# Celda 4
# Eliminar columnas con más del 25% de nulos
...

# Imputar el resto con la media
...

# Escalado
...


In [ ]:
# Celda 5 Train, test split. Atención al balanceo de la variable target.
...

# Aplicar SMOTE solo al conjunto de entrenamiento
...

# Verificar balance
...

## Entrenamiento de modelos clásicos

In [ ]:

# Celda 6
# Regresión Logística, fit y predicciones
...

print("Logistic Regression F1 Score:", f1_score(y_test, lr_preds))

## Entrenamiento de red neuronal (MLP)

In [ ]:
# Celda 7. Crea una red neuronal, prueba con distintas combinaciones de capas
model = Sequential()
model.add(...)
# Utiliza la red de salida adecuada (clasificación binaria)
model.add(...)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Celda 8

# Por su desbalanceo, usaremos los pesos de clase calculados automáticamente
weights = class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights = dict(zip(np.unique(y_train), weights))
print("Pesos de clase:", class_weights)

early_stopping = EarlyStopping(
    monitor=...,
    patience=...,
    restore_best_weights=...,
    verbose=...
)

# Entrenamiento del modelo con class_weight
history = model.fit(
    ...,
    ...,
    validation_data=(X_test, y_test),
    epochs=...,
    batch_size=...,
    class_weight=...,
    callbacks=[...],
    verbose=1
)


## Evaluación de resultados y comparación

In [ ]:
# Celda 9
# Lanza predicciones y después define un umbral a partir del cual predecimos una clase como positiva
y_pred_prob = model.predict(X_test)
y_pred_class = (y_pred_prob > ...).astype(int)

print("Matriz de Confusión:")
print(...)

print("\nReporte de Clasificación:")
print(...)


In [ ]:
# Celda 10
# Curvas de entrenamiento
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.title("Curva de Aprendizaje")
plt.show()

In [ ]:
# Celda 11
precision, recall, thresholds = precision_recall_curve(y_test, y_pred_prob)

plt.figure(figsize=(8, 5))
plt.plot(recall, precision, marker='.')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Curva Precision-Recall')
plt.grid(True)
plt.show()

## Conclusiones


**Resumen de lo aprendido:**
- Hemos entrenado una red neuronal simple y modelos clásicos sobre datos industriales reales.
- El dataset SECOM tiene ruido y desequilibrios que afectan el rendimiento, pero la red neuronal ha logrado resultados mucho mejores que la regresión.
- La preparación de datos (tratamiento de nulos, escalado, SMOTE) ha sido esencial para lograr buenos resultados.

**Próximos pasos sugeridos:**
1. Ajustar los hiperparámetros de la red neuronal con búsqueda en grid.
3. Probar arquitecturas más profundas o usar técnicas de regularización como dropout.

**Licencia del dataset:** SECOM Manufacturing Data Set — Fuente: UCI ML Repository, Licencia: [CC BY 4.0](https://creativecommons.org/licenses/by/4.0/)
